In [1]:
import pandas as pd
import pymysql
import time
import os

In [2]:
path='data/'

In [6]:
kp=[]
for i in os.listdir(path):
    sz=os.stat(path+i).st_size/1024/1024.0
#     print i.rjust(17),sz
    if sz>1.5:
        kp.append(i)
len(kp)

86

### diary3

In [20]:
diary_id = 3


In [19]:
def getDF(diary_id):
    diary_map={1:'diary_2',2:'diary_3',3:'diary_8',4:'diary_9'}
    fl = [i for i in os.listdir(path) if i.startswith(diary_map[diary_id])]
#     print len(fl)
    fl = [i for i in fl if os.stat(path+i).st_size/1024/1024.0>1.5]
#     print len(fl)
    
    coll=[]
    for i in fl:
        df0 = pd.read_csv(path+i)
        df0['serial']=int(i.split('_')[-1].split('.csv')[0])
        coll.append(df0)
        del df0
    print len(coll)

    df = pd.concat(coll,axis=0)
    print df.shape
    print 'remove null post_id'
    df = df[(~df['post_ID'].isnull())]
    print df.shape    
    
    seq = sorted(df['serial'].unique())
    date = ['2019-02-21','2018-02-22','2018-02-23','2018-02-24','2018-02-25']
    times = {i:j for i,j in zip(seq,date)}
    df['date']=df['serial'].map(times)
    
    df['post_ID'] = df['post_ID'].astype('int')
    print 'drop duplicates'
    print df.shape
    df = df.drop_duplicates()
    print df.shape
    df = df[['post_ID','date','comments','favor','views']]
    
    return df


    

### post_ID is not null

In [17]:
from secret import dburl, dbusername, dbpasswd
from time import time

db = pymysql.connect(host=dburl,    # your host, usually localhost
                     user=dbusername,         # your username
                     passwd=dbpasswd,  # your password
                     port=3306,
                     db="research",
                     charset='utf8')

cur=db.cursor()

In [18]:
def BatchInsert(df):
    
    df=df[['post_ID','date','comments','favor','views']]
    batch = [df.iloc[n*10000:(n+1)*10000,:] for n in range(0 ,int(df.shape[0]/10000.0)+1)]
    
    start=time()
    n=0
    for d in batch:
        
        sql = """insert ignore into monitorpost values {}""".format(str(map(lambda x: tuple(x),d.values))[1:-1])
        n+=1
        cur.execute(sql)
        db.commit()
        print n,'{:.2f}s |'.format(time()-start),
        start=time()

    

In [21]:
%%time
df = getDF(3)
print df.shape

20
(2851714, 5)
remove null post_id
(592347, 5)
drop duplicates
(592347, 6)
(592347, 6)
(592347, 5)
CPU times: user 1.02 s, sys: 228 ms, total: 1.25 s
Wall time: 1.34 s


In [22]:
%%time
BatchInsert(df)

1 1.85s | 2 1.88s | 3 1.78s | 4 1.71s | 5 1.83s | 6 1.71s | 7 1.80s | 8 1.84s | 9 1.74s | 10 1.77s | 11 1.94s | 12 1.89s | 13 1.81s | 14 1.73s | 15 1.75s | 16 1.73s | 17 1.76s | 18 1.77s | 19 1.75s | 20 1.77s | 21 1.75s | 22 1.76s | 23 1.78s | 24 1.75s | 25 1.75s | 26 1.74s | 27 1.72s | 28 1.74s | 29 1.75s | 30 1.75s | 31 1.76s | 32 1.76s | 33 1.80s | 34 1.77s | 35 1.97s | 36 1.76s | 37 1.81s | 38 1.85s | 39 1.80s | 40 1.78s | 41 1.80s | 42 1.72s | 43 1.76s | 44 1.74s | 45 1.72s | 46 1.81s | 47 1.79s | 48 1.78s | 49 1.73s | 50 1.78s | 51 1.79s | 52 1.77s | 53 1.76s | 54 1.77s | 55 1.78s | 56 1.75s | 57 1.76s | 58 1.71s | 59 1.72s | 60 0.44s |CPU times: user 1.67 s, sys: 56 ms, total: 1.73 s
Wall time: 1min 45s

